# SCIALOG Statistical Analyses

Steps 1 - 3

In [ ]:
# Run this in a Jupyter notebook cell

import pandas as pd
from sklearn.metrics import cohen_kappa_score
import matplotlib.pyplot as plt
import seaborn as sns

def process_flat_sheet(filepath, coder_name):
    xls = pd.read_excel(filepath, sheet_name=None)
    all_dfs = []
    for sheet_name, df in xls.items():
        df.columns = [str(c).strip().lower() for c in df.columns]
        if not {'video', 'timestamp', 'speaker'}.issubset(df.columns):
            continue
        for i in range(4):
            annot_idx = 3 + i * 2
            if annot_idx >= len(df.columns):
                continue
            code_col = df.columns[annot_idx]
            temp = df[['video', 'timestamp', 'speaker', code_col]].copy()
            temp.columns = ['video', 'timestamp', 'speaker', 'annotation']
            temp['coder'] = coder_name
            temp['sheet'] = sheet_name
            all_dfs.append(temp.dropna(subset=['annotation']))
    return pd.concat(all_dfs, ignore_index=True)

# Load your files
df_max = process_flat_sheet("data/sampled video verification (Max).xlsx", "max")
df_evey = process_flat_sheet("data/sampled video verification (Evey).xlsx", "evey")

# Merge and compare
df_compare = df_max.merge(df_evey, on=["video", "timestamp"], suffixes=('_max', '_evey'))
kappa = cohen_kappa_score(df_compare['annotation_max'], df_compare['annotation_evey'])

print(f"Cohen's Kappa (overall): {kappa:.3f}")

# Frequency plots
sns.countplot(data=df_max, y='annotation')
plt.title("Max's Annotation Frequency")
plt.show()


: 